# Pittsburgh-area aviation waypoints

In [1]:
import requests

### Fetch raw waypoints from the FAA

In [2]:
waypoints = []

# First 1,000 waypoints
search_data = {
    'dataType': 'LIDFIXESWAYPOINTS',
    'start': 0,
    'length': 1000,
    'sortcolumn': 'fix_identifier',
    'sortdir': 'asc',
    'searchval': 'PENNSYLVANIA',
    'r': 'NTAg7a9ZdvAPgPALayuxIQYO',
}

response = requests.post('https://nfdc.faa.gov/nfdcApps/controllers/PublicDataController/getLidData', data=search_data)
response = response.json()
waypoints = waypoints + response['data']

# Next 1,000 waypoints
search_data['start'] = 1000
response = requests.post('https://nfdc.faa.gov/nfdcApps/controllers/PublicDataController/getLidData', data=search_data)
response = response.json()
waypoints = waypoints + response['data']

In [3]:
print([response['totalrows'], len(waypoints)])

[1651, 1651]


### Convert raw waypoints to dataframe